In [1]:
from get_quantum_nums import get_quantum_numbers
import hadron
import openparticle as op
import numpy as np

In [16]:
g = 1
L = 100
mg = 0.01

mq = 1.0

In [3]:
K = 1
Lambda_perp = 0
P_perp = 0

In [4]:
qnums = get_quantum_numbers(K, Lambda_perp)
qnums

,Helicity,n,n⟂
0,+1,0.5,0
1,+1,1.0,0
2,-1,0.5,0
3,-1,1.0,0


In [5]:
pion = hadron.Hadron('meson', K)
pion.display()

<IPython.core.display.Latex object>

In [6]:
pion_fock = pion.get_states(K, P_perp, Lambda_perp)
pion_fock.display()

<IPython.core.display.Latex object>

In [7]:
def pplus(n):
    return 2 * np.pi * n / L

In [8]:
def Coeff_iG(q1, q2, q3, q4):
    n1, n2 = qnums.iloc[q1]['n'], qnums.iloc[q2]['n'] 
    n3, n4 = qnums.iloc[q3]['n'], qnums.iloc[q4]['n']

    h1, h2 = qnums.iloc[q1]['Helicity'], qnums.iloc[q2]['Helicity'] 
    h3, h4 = qnums.iloc[q3]['Helicity'], qnums.iloc[q4]['Helicity']
    
    return 4/((pplus(q1) - pplus(q4))**2 + mg**2) * np.sqrt(pplus(q1) * pplus(q2) *\
                       pplus(q3) * pplus(q4)) * (h1 == h4) * (h2 == h3)
    

In [14]:
H_iG = op.ParticleOperatorSum([])

for i in range(len(qnums)):
    for j in range(len(qnums)):
        for k in range(len(qnums)):
            for l in range(len(qnums)):
                if qnums.iloc[i]['n'] % 1 == 0.5 and qnums.iloc[j]['n'] % 1 == 0.5 and qnums.iloc[k]['n'] % 1 == 0.5 and qnums.iloc[l]['n'] % 1 == 0.5:
                c = Coeff_iG(i, j, k, l)
                op_st = 'b' + str(i) + '^ ' +\
                    'd' + str(j) + '^ ' +\
                    'd' + str(k) + ' ' +\
                    'b' + str(l)
                H_iG += (-g**2) * Coeff_iG(i, j, k, l) * op.ParticleOperator(op_st)

H_iG = H_iG.cleanup()


In [39]:
H_free = op.ParticleOperatorSum([])

for i in range(len(qnums)):
    if qnums.iloc[i]['n'] % 1 == 0.5:
        op_st_f = 'b' + str(i) + "^ " + 'b' + str(i) 
        op_st_af = 'd' + str(i) + "^ " + 'd' + str(i) 
        x = qnums.iloc[i]['n'] / K
        coeff = mq**2 / x
        H_free += coeff * op.ParticleOperator(op_st_f) + coeff * op.ParticleOperator(op_st_af)

H_free = H_free.cleanup()

In [41]:
H_free.display()

<IPython.core.display.Latex object>

In [15]:
H_iG.display()

<IPython.core.display.Latex object>

$U = e^{i2\phi H}$